# Test Age and Gender Models on a Custom Image
This notebook loads your trained models and predicts age and gender for an image in the `test` folder.

In [3]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp

# Model and image config
IMG_SIZE = (128, 128)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_DIR = '../models'
TEST_IMG_DIR = '../test'
MAX_AGE = 116 

# Initialize MediaPipe Face Detector
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

normalize_transform = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# List test images
test_images = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
print('Test images:', test_images)

ImportError: cannot import name 'runtime_version' from 'google.protobuf' (c:\Users\Pantat S\Documents\GitHub\project_face_analysis\env\Lib\site-packages\google\protobuf\__init__.py)

In [ ]:
class BaseCNN(nn.Module):
    def __init__(self, output_units=1, task='age'):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2), nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2), nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2), nn.BatchNorm2d(128),
            nn.Flatten()
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * (IMG_SIZE[0] // 8) * (IMG_SIZE[1] // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, output_units)
        )
        self.task = task

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        if self.task == 'gender':
            x = torch.sigmoid(x)
        return x

In [ ]:
# Load the correct model class for ResNet-based models
from torchvision import models
import torch.nn as nn


class ResNetModel(nn.Module):
    def __init__(self, output_units=9, task='age'):
        super().__init__()
        self.resnet = models.resnet18(weights=None)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, output_units)
        self.task = task

    def forward(self, x):
        return self.resnet(x)

# Instantiate models
age_model = ResNetModel(output_units=9, task='age')
gender_model = ResNetModel(output_units=1, task='gender')
emotion_model = ResNetModel(output_units=5, task='emotion')


# Instantiate models
age_model = ResNetModel(output_units=9, task='age')
gender_model = ResNetModel(output_units=1, task='gender')
emotion_model = ResNetModel(output_units=5, task='emotion') 

# Load weights
age_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'age_model_resnet.pth'), map_location=DEVICE))
gender_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'gender_model_resnet.pth'), map_location=DEVICE))
emotion_model.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'emotion_model_resnet.pth'), map_location=DEVICE))

age_model.eval()
age_model.to(DEVICE)
gender_model.eval()
gender_model.to(DEVICE)
emotion_model.eval()
emotion_model.to(DEVICE)


In [ ]:
# Block 4: Define transforms (should match validation/test transforms used in training)
test_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor()
])

# Gender label mapping (edit if needed)
gender_map = {0: 'Female', 1: 'Male'}

# Emotion label mapping
emotion_map = {0: 'Angry', 1: 'Happy', 2: 'Neutral', 3: 'Sad', 4: 'Surprise'}

In [ ]:
print("\nRunning predictions with MediaPipe face detection (age, gender, emotion)...")

# Define a single transform with normalization
test_transform_norm = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    normalize_transform
])

for fname in test_images:
    img_path = os.path.join(TEST_IMG_DIR, fname)

    # Read the image using OpenCV for MediaPipe compatibility
    img_cv2 = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)

    # Process the image with MediaPipe
    results = face_detection.process(img_rgb)

    if not results.detections:
        print(f"No face detected in {fname}. Skipping.")
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title(f'No face detected in {fname}')
        plt.show()
        continue

    first_detection = results.detections[0]
    bbox = first_detection.location_data.relative_bounding_box
    
    h, w, _ = img_cv2.shape
    x, y = int(bbox.xmin * w), int(bbox.ymin * h)
    box_w, box_h = int(bbox.width * w), int(bbox.height * h)

    x1, y1 = max(0, x), max(0, y)
    x2, y2 = min(w, x + box_w), min(h, y + box_h)
    
    cropped_face = img_rgb[y1:y2, x1:x2]
    
    if cropped_face.shape[0] < 10 or cropped_face.shape[1] < 10:
        print(f"Detected face in {fname} is too small or invalid after crop. Skipping.")
        continue

    pil_face = Image.fromarray(cropped_face)

    # Prepare a single, normalized tensor for all models
    img_tensor = test_transform_norm(pil_face).unsqueeze(0).to(DEVICE)

    # ---- AGE ----
    with torch.no_grad():
        age_logits = age_model(img_tensor)
        age_probs = torch.softmax(age_logits, dim=1)
        age_idx = int(torch.argmax(age_probs, dim=1).item())
        age_conf = float(torch.max(age_probs).item())

        # Map to actual bin label
        age_bins = ["0-2", "3-6", "7-12", "13-19", "20-29", "30-39", "40-49", "50-59", "60+"]
        age_label = age_bins[age_idx]

    # ---- GENDER ----
    with torch.no_grad():
        gender_logits = gender_model(img_tensor)
        gender_prob = torch.sigmoid(gender_logits).item()
        gender_idx = int(round(gender_prob))
        gender_label = gender_map[gender_idx]

    # ---- EMOTION ----
    with torch.no_grad():
        emotion_logits = emotion_model(img_tensor)
        emotion_probs = torch.softmax(emotion_logits, dim=1)
        emotion_idx = int(torch.argmax(emotion_probs, dim=1).item())
        emotion_label = emotion_map.get(emotion_idx, 'Unknown')

    # Draw results
    cv2.rectangle(img_cv2, (x, y), (x + box_w, y + box_h), (0, 255, 0), 2)
    cv2.putText(img_cv2, f'Age: {age_label} ({age_conf:.2f})', (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.putText(img_cv2, f'Gender: {gender_label}', (x, y - 35),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.putText(img_cv2, f'Emotion: {emotion_label}', (x, y - 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    plt.imshow(cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()